In [1]:
import numpy as np
import math
from tqdm import tqdm
import torch
import torchvision
import torch.nn as nn
from torch.utils import data
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import torchvision.transforms.functional as TF
from torch.optim import lr_scheduler
import time
import os
import cv2
import pdb
from PIL import Image
import matplotlib.pyplot as plt 
cuda = torch.cuda.get_device_name(0)
print(cuda)

GeForce RTX 2080 Ti


In [43]:
print(os.getcwd())

/home/ppant/Desktop/Pranshu/Lab/LES-DNS/SR-DNS_Net/k_21


In [36]:
if not os.path.exists('save_3c'):
    os.mkdir('save_3c')

if not os.path.exists('model_3c'):
    os.mkdir('model_3c')

if not os.path.exists('test_3c'):
    os.mkdir('test_3c')

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), -1, 64, 64)
    return x
    

## Train Set

In [3]:
nums = 100*100
image_paths = ["DNS-LES_3C/les_3c/"+str(i/5670)[:3]+"0/"+str(i%567)+".png" for i in range(nums)]
target_paths = ["DNS-LES_3C/dns_3c/"+str(i/5670)[:3]+"0/"+str(i%567)+".png" for i in range(nums)] 
print(len(image_paths))
#print(image_paths[170099])
nums = 300*100
image_paths_1 = ["DNS-LES_3C/les_3c/"+str(i/5670)[:3]+"0/"+str(i%567)+".png" for i in range(30000, 30000+nums)]
target_paths_1 = ["DNS-LES_3C/dns_3c/"+str(i/5670)[:3]+"0/"+str(i%567)+".png" for i in range(30000, 30000+nums)] 
print(len(image_paths))
image_paths.extend(image_paths_1)
target_paths.extend(target_paths_1)
print(len(image_paths))
print(image_paths[39999])

10000
10000
40000
DNS-LES_3C/les_3c/10.0/464.png


## Dev Set

In [4]:
nums = 1000
image_paths_dev = ["DNS-LES_3C/les_3c/"+str(i/5670)[:3]+"0/"+str(i%567)+".png" for i in range(10000, 10000+nums)]
target_paths_dev = ["DNS-LES_3C/dns_3c/"+str(i/5670)[:3]+"0/"+str(i%567)+".png" for i in range(10000, 10000+nums)] 
print(len(image_paths_dev))
#print(image_paths[170099])
print(image_paths_dev[0])

1000
DNS-LES_3C/les_3c/1.70/361.png


## Test Set

In [5]:
nums = 1000
image_paths_test = ["DNS-LES_3C/les_3c/"+str(i/5670)[:3]+"0/"+str(i%567)+".png" for i in range(20000, 20000+nums)]
target_paths_test = ["DNS-LES_3C/dns_3c/"+str(i/5670)[:3]+"0/"+str(i%567)+".png" for i in range(20000, 20000+nums)] 
print(len(image_paths_test))
#print(image_paths[170099])
print(image_paths_test[0])

1000
DNS-LES_3C/les_3c/3.50/155.png


## Image Transforms (Trial)

In [6]:
# img_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomResizedCrop(size = 32, scale=(0.75, 1.0), ratio=(0.8, 1.25)),
#     transforms.RandomPerspective(distortion_scale=0.25, p=0.3, interpolation=3),
#     transforms.ColorJitter(brightness=0.35, contrast=0.25, saturation=0, hue=0),
#     torchvision.transforms.RandomRotation(7),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])

## DataLoader

In [7]:
class MyDataset(data.Dataset):
    def __init__(self, image_paths, target_paths, transform=None):

        self.image_paths = image_paths
        self.target_paths = target_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        ip = Image.open(self.image_paths[index])
        op = Image.open(self.target_paths[index])
        x = self.transform(ip)
        y = self.transform(op)
        return x, y

## Model for SR-DNS_Net

In [8]:
class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            nn.LeakyReLU(inplace=True)
        )

In [9]:
class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1))
        layers.extend([
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

In [10]:
class MobileNetv2_SISR(nn.Module):
    def __init__(self):
        super(MobileNetv2_SISR, self).__init__()

        self.conv1 = nn.Sequential(
        nn.Conv2d(3, 32, 3, stride=1, padding=1),  # b, 32, 64, 64
        nn.BatchNorm2d(32),
        nn.LeakyReLU()
        )
        self.bottlenecks1 = nn.Sequential(
            InvertedResidual(32, 16, 1, 1),
            InvertedResidual(16, 16, 1, 1),
            InvertedResidual(16, 24, 1, 6),
            InvertedResidual(24, 24, 1, 6),
            InvertedResidual(24, 32, 1, 6),
            InvertedResidual(32, 32, 1, 6),
            InvertedResidual(32, 32, 1, 6),
            InvertedResidual(32, 64, 1, 6),
            InvertedResidual(64, 64, 1, 6),
            InvertedResidual(64, 64, 1, 6),
        )

        self.bottlenecks2 = nn.Sequential(
            InvertedResidual(64, 64, 1, 6),
            InvertedResidual(64, 96, 2, 6),
            InvertedResidual(96, 96, 1, 6),
            InvertedResidual(96, 96, 1, 6),
            InvertedResidual(96, 128, 1, 6),
            InvertedResidual(128, 128, 1, 6),
            InvertedResidual(128, 128, 1, 6),
            InvertedResidual(128, 256, 1, 6),
            #InvertedResidual(256, 256, 1, 6),
        )
        self.deconv1 = nn.Sequential(
        nn.ConvTranspose2d(64, 3, 3, stride=1, padding=1),  # b, 1, 64, 64
        #nn.Tanh()
        )
        self.pix_shuffle = nn.PixelShuffle(2)
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.conv1(x)
        
        out = self.bottlenecks1(out)
        int1 = out
        out = self.bottlenecks2(out)
        out = self.pix_shuffle(out)
        out = torch.add(out, int1)

        out = self.deconv1(out)
        out1 = self.tanh(out)+0.0001*out
        return out1

        #return out
        
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out')
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.ones_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.zeros_(m.bias)

## Evaluation Metrics
* PSNR
* SSIM
* KE
* Vorticity

In [11]:
def PSNR(op, t, batch_size): 
    mse = torch.sum((t - op) ** 2) 
    #print(mse.size())
    mse /= (batch_size*64*64)

    max_pixel = 1.
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    #print(psnr.size())
    return psnr 

In [12]:
def KE(img, op, t): 
    ke_recon = torch.sum(op ** 2)/op.size()[0] 
    ke_dns = torch.sum(t ** 2)/t.size()[0] 
    ke_les = torch.sum(img ** 2)/img.size()[0] 
 
    return ke_les, ke_recon, ke_dns

In [13]:
def Avg_KE(img, op, t): 
    # pdb.set_trace()

    op = np.squeeze(op)
    img = np.squeeze(img)
    t = np.squeeze(t)
    # pdb.set_trace()

    ke_recon = torch.mean(torch.abs(op - torch.mean(op)))
    ke_dns = torch.mean(torch.abs(t - torch.mean(t)))
    ke_les = torch.mean(torch.abs(img - torch.mean(img)))
 
    return ke_les, ke_recon, ke_dns

In [14]:
from skimage.measure import compare_ssim
def SSIM(op, t, batch_size):
    ssim = 0 
    #print(op.size(), t.size())
    for i in range(op.size()[0]):
        tar = to_img(t[i])
        out = to_img(op[i])
        # print(out[0,0].size())
        (score, diff) = compare_ssim(out[0,0].cpu().numpy(), tar[0,0].cpu().numpy(), full=True)
        ssim+=score/batch_size
    
        #print("SSIM: {}".format(score))
    return ssim

In [15]:
def train_epoch(model, data_loader, criterion, optimizer):
    model.train()
    
    running_loss = 0
    avg_psnr = 0
    avg_psnr_les = 0

    start_time = time.time()
    print('Train Loop')
    for batch_num, (img, target) in tqdm(enumerate(data_loader), total=len(data_loader), ascii=True):

        img = img.to(device)
        target = target.to(device)

        output = model(img)
        loss = criterion(output, target)
        psnr = PSNR(output, target, batch_size)
        psnr_les = PSNR(img, target, batch_size)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()/len(data_loader)
        avg_psnr += psnr.item()/len(data_loader)
        avg_psnr_les += psnr_les.item()/len(data_loader)

    #print(' ')
    print('Train_Loss:{:.6f}, PSNR_DNS:{:.4f}, PSNR_LES:{:.4f}'.format(running_loss, psnr, psnr_les))
    torch.cuda.empty_cache()
    end_time = time.time()
    del img
    del target
    del loss
    print("Train Time: {:.2f} s".format(end_time-start_time))

    return running_loss

In [16]:
def dev_epoch(model, data_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0
        avg_psnr = 0
        avg_psnr_les = 0
        avg_ssim_dns = 0
        avg_ssim_les = 0

        start_time = time.time()
        print('Dev Loop')
        print(' ')

        for batch_num, (img, target) in tqdm(enumerate(data_loader), total=len(data_loader), ascii=True):

            img = img.to(device)
            target = target.to(device)

            output = model(img)
            loss = criterion(output, target)
            psnr = PSNR(output, target,batch_size)
            psnr_les = PSNR(img, target,batch_size)
            ssim_dns = SSIM(output, target, batch_size)
            ssim_les = SSIM(img, target, batch_size)

            running_loss += loss.item()/len(data_loader)
            avg_psnr += psnr.item()/len(data_loader)
            avg_psnr_les += psnr_les.item()/len(data_loader)
            avg_ssim_dns += ssim_dns.item()/len(test_loader)
            avg_ssim_les += ssim_les.item()/len(test_loader)

        torch.cuda.empty_cache()
        end_time = time.time()

        del img
        del target
        del loss
        del psnr
        del psnr_les

        print("Dev Time: {:.2f} s".format(end_time-start_time))

        return running_loss, avg_psnr_les, avg_psnr, avg_ssim_les, avg_ssim_dns

## Load Datasets

In [17]:
num_epochs = 200
batch_size = 32
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.05),
    transforms.RandomVerticalFlip(p=0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = MyDataset(image_paths, target_paths, transform=img_transform)
train_loader_args = dict(batch_size=batch_size, shuffle=True, num_workers=8)
train_loader = data.DataLoader(train_dataset, **train_loader_args)
print(train_dataset.__len__())

40000


In [18]:
dev_dataset = MyDataset(image_paths_dev, target_paths_dev, transform=test_transform)
dev_loader_args = dict(batch_size=batch_size, shuffle=False, num_workers=8)
dev_loader = data.DataLoader(dev_dataset, **dev_loader_args)
print(dev_dataset.__len__())

1000


In [19]:
test_dataset = MyDataset(image_paths_test, target_paths_test, transform=test_transform)
test_loader_args = dict(batch_size=1, shuffle=False, num_workers=8)
test_loader = data.DataLoader(test_dataset, **test_loader_args)
print(test_dataset.__len__())

1000


## Load Model

In [20]:
model = MobileNetv2_SISR()
model.apply(init_weights)
device = torch.device("cuda")
model.eval()
model.to(device)        
print(model)

MobileNetv2_SISR(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (bottlenecks1): Sequential(
    (0): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.01, inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (

## Train Model

In [ ]:
num_epochs = 200
Train_Loss = []
Dev_Loss = []
Dev_Acc = []
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.1, patience=3, threshold=5e-4, eps=1e-6)

for epoch in range(num_epochs):

    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    dev_loss, psnr_les, psnr, ssim_les, ssim_dns = dev_epoch(model, dev_loader, criterion)
    Train_Loss.append(train_loss)
    Dev_Loss.append(dev_loss)

    scheduler.step(dev_loss)

    print(' ')
    print('epoch [{}/{}], Train_Loss:{:.6f}, Dev_Loss:{:.6f}'.format(epoch+1, num_epochs, train_loss, dev_loss))
    print('PSNR_DNS:{:.4f}, PSNR_LES:{:.4f}, SSIM_LES:{:.4f}, SSIM_DNS:{:.4f}'.format(psnr, psnr_les, ssim_les, ssim_dns))

    torch.save(model.state_dict(), 'model_3c/SISR_mv2f_{}.pth'.format(epoch))
    scheduler.step(train_loss)
    print(optimizer)
    print('='*100)


## Plots

In [28]:
def plot_MAE(L, R, D):

    MAE = np.abs((np.array(R) - np.array(D))/np.array(D))
    print(MAE.shape)
    # print(MAE)
    avg = np.mean(MAE)
    plt.xticks([])
    plt.title('Reconstruction')
    plt.ylabel('MAE')
    plt.ylim(0,1.2)
    plt.plot(MAE, 'ko', fillstyle = 'none')
    plt.axhline(avg, color = 'r')
    plt.savefig('MAE.eps')
    plt.close()

    MAE_L = np.abs((np.array(L) - np.array(D))/np.array(D))
    print(MAE.shape)
    # print(MAE)
    avg_l = np.mean(MAE_L)
    plt.xticks([])
    plt.title('LES')
    plt.ylabel('MAE')
    # plt.plot(MAE, 'yo', fillstyle = 'none')
    plt.plot(MAE_L, 'ko', fillstyle = 'none')
    # plt.axhline(avg, color = 'r')
    plt.axhline(avg_l, color = 'r')
    plt.savefig('MAE_L.eps')  # plt.show()
    plt.close()

In [29]:
def plot_Avg_MAE(L, R, D):

    # MAE = np.abs((np.array(R) - np.array(D))/np.array(D))
    # print(MAE.shape)
    # print(MAE)
    avg = np.mean(np.abs(R))
    # plt.xticks([])
    plt.title('Reconstruction')
    plt.ylabel('Average Turbulent Velocity')
    # plt.ylim(0,1.2)
    plt.plot(np.abs(R), 'ko', fillstyle = 'none')
    plt.axhline(avg, color = 'r')
    plt.savefig('AM.eps')
    plt.close()

    # MAE_L = np.abs((np.array(L) - np.array(D))/np.array(D))
    # print(MAE.shape)
    # print(MAE)
    avg_l = np.mean(np.abs(D))
    # plt.xticks([])
    plt.title('DNS')
    plt.ylabel('Average Turbulent Velocity')
    # plt.plot(MAE, 'yo', fillstyle = 'none')
    plt.plot(np.abs(D), 'ko', fillstyle = 'none')
    # plt.axhline(avg, color = 'r')
    plt.axhline(avg_l, color = 'r')
    plt.savefig('AM_L.eps')  # plt.show()
    plt.close()

    error = np.abs(np.array(D) - np.array(R))
    avg_e = np.mean(np.abs(error))
    # plt.xticks([])
    plt.title('Error')
    plt.ylim(0, 0.35)
    plt.ylabel('Turbulent Velocity Error')
    # plt.plot(MAE, 'yo', fillstyle = 'none')
    plt.plot(error, 'ko', fillstyle = 'none')
    # plt.axhline(avg, color = 'r')
    plt.axhline(avg_e, color = 'r')
    plt.savefig('error.eps')  # plt.show()
    plt.close()

In [31]:
#combined plot for turbulent velocity
def plot_Avg_MAE(L, R, D):

    # MAE = np.abs((np.array(R) - np.array(D))/np.array(D))
    # print(MAE.shape)
    # print(MAE)
    avg = np.mean(np.abs(R))
    # plt.xticks([])
    # plt.title('Reconstruction')
    plt.ylabel('Average Turbulent Velocity')
    plt.xlabel('Samples')
    # plt.ylim(0,1.2)
    plt.plot(np.abs(R),  marker = '^', c = 'dodgerblue', label='Recon',ls=' ', ms='3.5')


    # MAE_L = np.abs((np.array(L) - np.array(D))/np.array(D))
    # print(MAE.shape)
    # print(MAE)
    avg_l = np.mean(np.abs(D))
    # plt.xticks([])
    # plt.title('DNS')
    # plt.ylabel('Average Turbulent Velocity')
    # plt.plot(MAE, 'yo', fillstyle = 'none')
    plt.plot(np.abs(D), marker = '+', c = 'darkorange', label='DNS',ls=' ', ms='4')
    # plt.axhline(avg, color = 'r')
    plt.axhline(avg_l, color = 'red', ls='-.',label='Avg. DNS', lw='1.5')
    plt.axhline(avg, color = 'k', ls='-.', label='Avg. Recon', lw='1.5')
    plt.legend(loc='best')
    plt.savefig('combined.eps')  # plt.show()
    plt.close()


## Metric Predictions (Test and Train)

In [39]:
def test_predictions(model, test_loader):
    with torch.no_grad():
        model.eval()
        avg_psnr = 0
        avg_psnr_les = 0
        avg_dns_ke = 0
        avg_les_ke = 0
        avg_recon_ke = 0
        avg_ssim_les = 0
        avg_ssim_dns = 0

        P = []
        L = []
        R = []
        D = []

        L_ = []
        R_ = []
        D_ = []

        for batch_idx, (img, target) in enumerate(test_loader):   
            
            img = img.to(device)
            target = target.to(device)
            
            out = model(img)
            psnr = PSNR(out, target, batch_size)
            psnr_les = PSNR(img, target, batch_size)

            les_ke, recon_ke, dns_ke = KE(img, out, target)
            les_ake, recon_ake, dns_ake = Avg_KE(img, out, target)

            ssim_dns = SSIM(out, target, batch_size)
            ssim_les = SSIM(img, target, batch_size)
            L.append(les_ke.cpu().numpy())
            R.append(recon_ke.cpu().numpy())
            D.append(dns_ke.cpu().numpy())

            L_.append(les_ake.cpu().numpy())
            R_.append(recon_ake.cpu().numpy())
            D_.append(dns_ake.cpu().numpy())

            pic = to_img(out.cpu()) #Only the first image of the batch.
            save_image(pic, 'test_3c/image_{}.png'.format(batch_idx))
            t = to_img(target) #Only the first image of the batch.
            save_image(t, 'test_3c/image_{}_t.png'.format(batch_idx))
            i = to_img(img) #Only the first image of the batch.
            save_image(i, 'test_3c/image_{}_og.png'.format(batch_idx))

            del img
            del target

            avg_psnr += psnr.item()/len(test_loader)
            avg_psnr_les += psnr_les.item()/len(test_loader)

            avg_dns_ke += dns_ke.item()/len(test_loader)
            avg_recon_ke += recon_ke.item()/len(test_loader)
            avg_les_ke += les_ke.item()/len(test_loader)

            avg_ssim_les += ssim_les.item()/len(test_loader)
            avg_ssim_dns += ssim_dns.item()/len(test_loader)

    plot_MAE(L, R, D)
    plot_Avg_MAE(L_, R_, D_)
    return avg_psnr, avg_psnr_les, avg_les_ke, avg_recon_ke, avg_dns_ke, avg_ssim_les, avg_ssim_dns

In [40]:
batch_size = 1
avg_psnr, avg_psnr_les, avg_les_ke, avg_recon_ke, avg_dns_ke, avg_ssim_les, avg_ssim_dns = test_predictions(model, test_loader)
print(avg_psnr)
print(avg_psnr_les)
print(avg_recon_ke)
print(avg_dns_ke)
print(avg_les_ke)
print(avg_ssim_les)
print(avg_ssim_dns)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
(1000,)
(1000,)
14.990708944320705
11.733874299049377
1341.4863097534167
1519.8945644531243
2152.8065732421906
0.8105047876127578
0.8720655256391564


In [26]:
def train_predictions(model, train_loader):
    with torch.no_grad():
        model.eval()
        avg_psnr = 0
        avg_psnr_les = 0
        avg_dns_ke = 0
        avg_les_ke = 0
        avg_recon_ke = 0
        avg_ssim_les = 0
        avg_ssim_dns = 0

        P = []

        for batch_idx, (img, target) in enumerate(train_loader):  

            # if batch_idx > 3:
            #     break 
            
            img = img.to(device)
            target = target.to(device)
            
            out = model(img)
            psnr = PSNR(out, target, batch_size)
            psnr_les = PSNR(img, target, batch_size)
            les_ke, recon_ke, dns_ke = KE(img, out, target)
            ssim_dns = SSIM(out, target, batch_size)
            ssim_les = SSIM(img, target, batch_size)

            pic = to_img(out.cpu()) #Only the first image of the batch.
            save_image(pic, 'test_3c/image_tr_{}.png'.format(batch_idx))
            t = to_img(target) #Only the first image of the batch.
            save_image(t, 'test_3c/image_tr_{}_t.png'.format(batch_idx))
            i = to_img(img) #Only the first image of the batch.
            save_image(i, 'test_3c/image_tr_{}_og.png'.format(batch_idx))

            del img
            del target

            avg_psnr += psnr.item()/len(train_loader)
            avg_psnr_les += psnr_les.item()/len(train_loader)

            avg_dns_ke += dns_ke.item()/len(train_loader)
            avg_recon_ke += recon_ke.item()/len(train_loader)
            avg_les_ke += les_ke.item()/len(train_loader)

            avg_ssim_les += ssim_les.item()/len(train_loader)
            avg_ssim_dns += ssim_dns.item()/len(train_loader)

    return avg_psnr, avg_psnr_les, avg_les_ke, avg_recon_ke, avg_dns_ke, avg_ssim_les, avg_ssim_dns

In [ ]:
batch_size = 32
avg_psnr, avg_psnr_les, avg_les_ke, avg_recon_ke, avg_dns_ke, avg_ssim_les, avg_ssim_dns = train_predictions(model, train_loader)
print(avg_psnr)
print(avg_psnr_les)
print(avg_recon_ke)
print(avg_dns_ke)
print(avg_les_ke)
print(avg_ssim_les)
print(avg_ssim_dns)

## Use this model for loading...

In [20]:
model = MobileNetv2_SISR()
PATH = 'model_3c/SISR_mv2f_48.pth'
model.load_state_dict(torch.load(PATH))
device = torch.device("cuda")
model.eval()
model.to(device)

MobileNetv2_SISR(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (bottlenecks1): Sequential(
    (0): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.01, inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (